In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Load training and testing datasets
train_df = pd.read_csv("dataset.txt")
test_df = pd.read_csv("test.txt")

# Combine for consistent label encoding
combined = pd.concat([train_df, test_df])

# Encode categorical columns
for col in ["Age Group", "Gender", "Target Class"]:
    le = LabelEncoder()
    combined[col] = le.fit_transform(combined[col])

# Split them back after encoding
train_df = combined.iloc[:len(train_df)]
test_df = combined.iloc[len(train_df):]

# Separate features and targets
X_train = train_df.drop(columns=["ID", "Target Class"])
y_train = train_df["Target Class"]

X_test = test_df.drop(columns=["ID", "Target Class"])
y_test = test_df["Target Class"]


In [2]:

from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00


In [3]:
import joblib
joblib.dump(model, "knee_classifier.pkl")


['knee_classifier.pkl']

In [4]:
import joblib
import numpy as np

# Load model
model = joblib.load("knee_classifier.pkl")

# Encodings
age_map = {'Young': 0, 'Young Adult': 1, 'Adult': 2, 'Older Adult': 3, 'Senior': 4}
gender_map = {'Male': 1, 'Female': 0, 'Non-binary': 2}

# Example input
input_data = {
    "Age Group": "Adult",
    "Height (cm)": 176,
    "Weight (kg)": 75,
    "Gender": "Male",
    "Temperature (°C)": 38.3,
    "Pressure (N)": 720,
    "Gyro X (rad/s)": 0.6,
    "Gyro Y (rad/s)": 0.4,
    "Gyro Z (rad/s)": 0.3,
    "Accel X (m/s²)": 1.3,
    "Accel Y (m/s²)": 1.0,
    "Accel Z (m/s²)": 9.5
}

# Preprocessing
input_vector = np.array([[ 
    age_map[input_data["Age Group"]],
    input_data["Height (cm)"],
    input_data["Weight (kg)"],
    gender_map[input_data["Gender"]],
    input_data["Temperature (°C)"],
    input_data["Pressure (N)"],
    input_data["Gyro X (rad/s)"],
    input_data["Gyro Y (rad/s)"],
    input_data["Gyro Z (rad/s)"],
    input_data["Accel X (m/s²)"],
    input_data["Accel Y (m/s²)"],
    input_data["Accel Z (m/s²)"]
]])

# Predict
prediction = model.predict(input_vector)
result = "Healthy Knee" if prediction[0] == 0 else "Potentially Unhealthy Knee"
print("Prediction:", result)

# Medical reasoning
def diagnose_knee(data):
    reasons = []
    if data["Age Group"] in ["Older Adult", "Senior"] and data["Pressure (N)"] > 700:
        reasons.append("Possible Osteoarthritis due to age and high joint pressure.")
    if data["Temperature (°C)"] > 37.5:
        reasons.append("Possible inflammation or infection due to elevated body temperature.")
    if abs(data["Gyro X (rad/s)"]) > 0.5 or abs(data["Gyro Y (rad/s)"]) > 0.5:
        reasons.append("Possible Meniscus Tear from abnormal rotation.")
    if abs(data["Accel Z (m/s²)"] - 9.8) > 1.5:
        reasons.append("Possible Ligament Strain due to unusual vertical impact.")
    return reasons

# Show possible conditions
if result == "Potentially Unhealthy Knee":
    diagnosis = diagnose_knee(input_data)
    print("Possible Conditions:")
    for issue in diagnosis:
        print("-", issue)


Prediction: Potentially Unhealthy Knee
Possible Conditions:
- Possible inflammation or infection due to elevated body temperature.
- Possible Meniscus Tear from abnormal rotation.


C:\Users\prajw\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
